In [10]:
import os
import sys
import torch
import psutil
import platform
import time

print("Machine Type:", platform.machine())

# Print current working directory
print("Current Working Directory:", os.getcwd())

# Print Python version
print("Python Version:", sys.version)

# Print PyTorch version
print("PyTorch Version:", torch.__version__)

# Check for CUDA and MPS availability
cuda_available = torch.cuda.is_available()
mps_available = hasattr(torch.backends, "mps") and torch.backends.mps.is_available()

if cuda_available:
    device = "cuda"
elif mps_available:
    device = "mps"
else:
    device = "cpu"

print("Computing Device:", device)
print("CUDA Available:", cuda_available)
print("MPS Available:", mps_available)

# Display CPU information
cpu_count = psutil.cpu_count(logical=True)
print("CPU Count:", cpu_count)

if hasattr(psutil, "cpu_freq"):
    cpu_freq = psutil.cpu_freq()
    if cpu_freq:
        print("CPU Frequency (MHz): Current = {:.2f}, Min = {:.2f}, Max = {:.2f}".format(
            cpu_freq.current, cpu_freq.min, cpu_freq.max))

# Display Memory information
mem = psutil.virtual_memory()
total_memory_gb = mem.total / (1024 ** 3)
available_memory_gb = mem.available / (1024 ** 3)
print("Total Memory: {:.2f} GB".format(total_memory_gb))
print("Available Memory: {:.2f} GB".format(available_memory_gb))

# Baseline Compute Time Test
def benchmark_pytorch(device, size=1000):
    """Measures time to perform matrix multiplication on the given device."""
    torch.manual_seed(42)
    A = torch.randn(size, size, device=device)
    B = torch.randn(size, size, device=device)

    # Warm-up
    for _ in range(5):
        _ = torch.mm(A, B)

    torch.cuda.synchronize() if device == "cuda" else None  # Ensure synchronization for CUDA

    start_time = time.time()
    for _ in range(10):  
        _ = torch.mm(A, B)
    torch.cuda.synchronize() if device == "cuda" else None  # Ensure synchronization for CUDA
    end_time = time.time()

    avg_time = (end_time - start_time) / 10
    print(f"Matrix Multiplication (Size {size}x{size}) on {device}: {avg_time:.6f} sec")

# Run benchmark test on available devices
benchmark_pytorch("cpu")
if cuda_available:
    benchmark_pytorch("cuda")
if mps_available:
    benchmark_pytorch("mps")

Machine Type: arm64
Current Working Directory: /Users/lyeeonardo/Downloads
Python Version: 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:20:01) [Clang 18.1.8 ]
PyTorch Version: 2.5.1
Computing Device: mps
CUDA Available: False
MPS Available: True
CPU Count: 14
CPU Frequency (MHz): Current = 4.00, Min = 1.00, Max = 4.00
Total Memory: 48.00 GB
Available Memory: 21.21 GB
Matrix Multiplication (Size 1000x1000) on cpu: 0.001158 sec
Matrix Multiplication (Size 1000x1000) on mps: 0.000023 sec
